In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes sympy tqdm google-colab

In [18]:
from datasets import get_dataset_config_names, load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, TrainerCallback, EarlyStoppingCallback
import shutil
from peft import LoraConfig, get_peft_model
import torch
from tqdm import tqdm
import re
from google.colab import drive
import os
import sympy
import gc

In [ ]:
drive.mount('/content/drive')

In [ ]:
save_dir = "/content/drive/MyDrive/Colab_Notebooks/CSE595_Proj/math-sft-models"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
configs = get_dataset_config_names("EleutherAI/hendrycks_math")
def load_all(split):
    return concatenate_datasets([load_dataset("EleutherAI/hendrycks_math", c, split=split) for c in configs])

train_ds = load_all("train")
test_ds = load_all("test")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

algebra/train-00000-of-00001.parquet:   0%|          | 0.00/505k [00:00<?, ?B/s]

algebra/test-00000-of-00001.parquet:   0%|          | 0.00/353k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1744 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/871 [00:00<?, ? examples/s]

precalculus/train-00000-of-00001.parquet:   0%|          | 0.00/354k [00:00<?, ?B/s]

precalculus/test-00000-of-00001.parquet:   0%|          | 0.00/242k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/746 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/546 [00:00<?, ? examples/s]

In [ ]:
SYSTEM_PROMPT = "You are a math assistant. Solve the problem step by step, explain your reasoning, and box the final answer using \\boxed{}."

def add_messages(example):
    example["messages"] = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": example["problem"]},
        {"role": "assistant", "content": example["solution"]}
    ]
    return example

train_ds = train_ds.map(add_messages)
test_ds  = test_ds.map(add_messages)
val_ds = test_ds.select(range(100))

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_8bit=True,
    torch_dtype=torch.float16
)

lora_config = LoraConfig(
    r=32, lora_alpha=16, target_modules=["q_proj", "v_proj"], lora_dropout=0.05
)
model = get_peft_model(model, lora_config)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    texts = [
        f"<|system|>\n{SYSTEM_PROMPT}<|end|>\n<|user|>\n{msg[1]['content']}<|end|>\n<|assistant|>\n{msg[2]['content']}<|end|>"
        for msg in examples["messages"]
    ]
    tokenized = tokenizer(texts, truncation=True, max_length=1024, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train = train_ds.map(tokenize_function, batched=True, remove_columns=train_ds.column_names)
tokenized_val = val_ds.map(tokenize_function, batched=True, remove_columns=val_ds.column_names)

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [24]:
def extract_boxed(latex_string):
    match = re.search(r'\boxed\s*\{', latex_string, re.IGNORECASE)

    if not match:
        return None

    # Get the index immediately after the opening brace
    start_index = match.end()

    # 2. Initialize the state variables
    brace_count = 1  # We start inside the first (outermost) brace
    content = []

    # 3. Iterate through the string from the start_index
    for i in range(start_index, len(latex_string)):
        char = latex_string[i]

        if char == '{':
            # Found an inner opening brace, increment counter
            brace_count += 1
            content.append(char)

        elif char == '}':
            # Found a closing brace, decrement counter
            brace_count -= 1

            if brace_count == 0:
                # This is the closing brace for the initial \boxed{
                return "".join(content)
            else:
                # It's an inner closing brace, keep appending
                content.append(char)

        else:
            # Append all other characters (content)
            content.append(char)

    # If the loop finishes without brace_count reaching zero, the braces were malformed
    return None

def normalize_sympy(s):
    if not s:
        return None
    try:
        # Attempt to simplify using sympy
        return sympy.sympify(s)
    except (sympy.SympifyError, TypeError): # Catch specific sympy errors and potential type errors
        # If sympy fails, return None. We won't do string cleaning here.
        return None

def compute_em(eval_pred):
    predictions, _ = eval_pred  # predictions: (num_samples, seq_len)
    correct = 0
    batch_size = 4

    # Ensure val_ds is accessible if not passed directly (it's a global in this notebook context)
    # You might need to pass val_ds to compute_em or ensure it's globally available
    # For now, assuming val_ds is globally accessible as in the original notebook
    # if 'val_ds' not in globals():
    #     raise NameError("val_ds not found. Ensure the validation dataset is loaded and named 'val_ds'.")


    for i in range(0, len(predictions), batch_size):
        batch_preds = predictions[i:i+batch_size]  # (batch, seq_len)
        batch_texts = [
            tokenizer.decode(p.tolist(), skip_special_tokens=True) # tokenizer should be globally accessible
            for p in batch_preds
        ]
        # Accessing val_ds globally - adjust if val_ds scope changes
        batch_golds = [ex["solution"] for ex in val_ds.select(range(i, i+batch_size))]


        for pred_text, gold_text in zip(batch_texts, batch_golds):
            pred_ans_str = extract_boxed(pred_text)
            gold_ans_str = extract_boxed(gold_text)

            pred_ans_sym = normalize_sympy(pred_ans_str)
            gold_ans_sym = normalize_sympy(gold_ans_str)

            # Compare normalized SymPy objects if both are successfully normalized
            # Also handle cases where both extracted strings are empty (e.g., no boxed answer)
            if pred_ans_sym is not None and gold_ans_sym is not None and pred_ans_sym == gold_ans_sym:
                correct += 1
            elif pred_ans_str == "" and gold_ans_str == "": # Consider empty extractions as a match if both are empty
                 correct += 1


        # Clear
        del batch_preds, batch_texts, batch_golds, pred_ans_str, gold_ans_str, pred_ans_sym, gold_ans_sym
        gc.collect()
        torch.cuda.empty_cache()

    # Need to handle potential discrepancy if len(predictions) is not the true number of examples evaluated
    # The predictions length should match the number of examples in the evaluation dataset batch
    # Let's assume len(predictions) is correct for now, but this is a point to verify.
    # The Trainer typically passes predictions corresponding to the eval_dataset.
    return {"em": correct / len(predictions)}

In [25]:
# Assuming extract_boxed and normalize_sympy from cell_id: H8xr_dwV211G are defined and correct
print(extract_boxed("djlkask \boxed {1/2+3} das \boxed {345}"))
print(normalize_sympy("0.5") == normalize_sympy("1/2"))

# You can add more test cases here if needed
print(extract_boxed("Some text without a box \boxed{1/2+3}"))
print(normalize_sympy("1 + 1"))
print(normalize_sympy("2"))
print(normalize_sympy("invalid math string"))

None
False
None
2
2
None


In [ ]:
class EMPrintCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        print(f"Step {state.global_step} | Val EM: {metrics.get('eval_em', 0):.1%}")

class DriveCheckpointCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        checkpoint_dir = f"checkpoint-{state.global_step}"
        local_path = os.path.join(args.output_dir, checkpoint_dir)
        drive_path = os.path.join(save_dir, checkpoint_dir)
        shutil.copytree(local_path, drive_path, dirs_exist_ok=True)

callbacks = [EMPrintCallback(), DriveCheckpointCallback(), EarlyStoppingCallback(3)]

In [ ]:
# Remove from TrainingArguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab_Notebooks/CSE595_Proj/math-sft-models/checkpoints",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    warmup_steps=50,
    lr_scheduler_type="cosine",
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=100,
    save_steps=300,
    save_total_limit=2,
    metric_for_best_model="eval_em",
    greater_is_better=True,
    load_best_model_at_end=True,
    fp16=False,
    bf16=True,
    report_to="none",
    dataloader_num_workers=1,
)

# Set in Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_em,
    callbacks=callbacks,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss


TypeError: argument 'ids': 'list' object cannot be interpreted as an integer

In [ ]:
best_checkpoint = trainer.state.best_model_checkpoint
final_drive_path = f"{save_dir}/final-model"
shutil.copytree(best_checkpoint, final_drive_path, dirs_exist_ok=True)
print(f"Final model saved to: {final_drive_path}")

In [ ]:
model = AutoModelForCausalLM.from_pretrained(final_drive_path, device_map="auto", load_in_4bit=True)

def test_em(num_samples=100):
    correct = 0
    for ex in test_ds.select(range(100, 100 + num_samples)):
        prompt = f"<|system|>\n{SYSTEM_PROMPT}<|end|>\n<|user|>\n{ex['problem']}<|end|>\n<|assistant|>"
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=512, do_sample=False)
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if normalize_sympy(extract_boxed(pred)) == normalize_sympy(extract_boxed(ex["solution"])):
            correct += 1
    return correct / num_samples

print(f"Test EM: {test_em(100):.1%}")

In [ ]:
# 加载你的 SFT 模型
model_name = "./tinyllama-sft"  # 或 GCS 路径
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

# 正则提取 \boxed{}
def extract_boxed(text):
    if not text:
        return ""
    m = re.search(r"\\boxed\{([^}]*)\}", text, re.DOTALL)
    return m.group(1).strip() if m else ""

# 标准化答案
def normalize_answer(s):
    if not s:
        return ""
    s = s.strip()
    s = re.sub(r'\s+', '', s)  # 去空格
    s = re.sub(r'\\boxed\{([^}]*)\}', r'\1', s)
    s = re.sub(r'[^0-9a-zA-Z\.\-\+\/]', '', s)
    return s.lower()

# 评估函数
def evaluate_exact_match(model, tokenizer, dataset, num_samples=100):
    correct = 0
    total = 0

    for ex in tqdm(dataset.select(range(num_samples))):
        problem = ex["problem"]
        gold_solution = ex["solution"]
        gold_ans = normalize_answer(extract_boxed(gold_solution))

        # 模型生成
        prompt = f"<|system|>\nYou are a math assistant. Solve step by step and box the final answer with \\boxed{{}}.\n<|user|>\n{problem}\n<|assistant|>"
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=False,
                temperature=0.0,
                pad_token_id=tokenizer.eos_token_id
            )

        pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # 提取 assistant 部分
        if "<|assistant|>" in pred_text:
            pred_text = pred_text.split("<|assistant|>")[-1]
        pred_ans = normalize_answer(extract_boxed(pred_text))

        if pred_ans == gold_ans:
            correct += 1
        total += 1

    return correct / total

# 运行评估
em_score = evaluate_exact_match(model, tokenizer, test_ds, num_samples=100)
print(f"Normalized Exact Match: {em_score:.1%}")